In [2]:
import numpy as np
import tensorflow as tf
import tensorflow.keras.layers as nn

from layer import attention, dice, AUGRU
from utils import sequence_mask

In [3]:
item_dim = 8
cate_dim = 6

In [11]:
# 构造虚拟数据
user_emb = np.random.rand(32,8+6)
item_join_emb  = np.random.rand(32,8+6)
hist_join_emb  = np.random.rand(32,20,8+6)
length = np.random.randint(1,21,(32,))

In [12]:
length

array([20, 13,  1,  5,  2,  5, 19,  4,  9,  1, 12, 15, 12, 12,  9,  2,  4,
       12, 20,  8, 15,  8, 15,  5, 20, 17, 16,  3, 13,  3, 12,  9])

In [13]:
item_join_emb = tf.cast(item_join_emb,tf.float32)
hist_join_emb = tf.cast(hist_join_emb,tf.float32)

In [14]:
dim_layers=[200,88,2]

In [15]:
fc = tf.keras.Sequential()
fc.add(nn.BatchNormalization())
for dim_layer in dim_layers[:-1]:
    fc.add(nn.Dense(dim_layer, activation='sigmoid'))
fc.add(nn.Dense(dim_layers[-1], activation=None))

In [16]:
# 第一次GRU
hist_gru_emb = nn.GRU(item_dim+cate_dim, return_sequences=True)(hist_join_emb) # (32, 20, 14)
# 利用第一次GRU得到的结果计算attention值，直接采用矩阵乘法进行计算向量之间的点积作为attention:(32,1,14)x(32,14,20)
tmp_attention = tf.matmul(tf.expand_dims(item_join_emb, 1), hist_gru_emb, transpose_b=True) # (32, 1, 20)
# 此处的(1,20)代表每一个user的被推荐的item与history序列（长度20）的item的相似度，一共20个相似度，将其作为attention
# 其中有效attention与history的有效长度相同，因此对其进行mask操作。此处不能用0-1mask,考虑到
# 后期要用到softmax 进行归一化，所以将无效值填充为负数无穷大。

In [17]:
# mask机制
hist_mask = tf.sequence_mask(length, max(length), dtype=tf.bool)
hist_mask = tf.expand_dims(hist_mask, axis=1)

paddings = tf.ones_like(tmp_attention) * (-2 ** 32 + 1)

tmp_attention = tf.where(hist_mask, tmp_attention, paddings)

In [18]:
hist_attn = tf.nn.softmax(tmp_attention)  # 32, 1, 20

In [19]:

hist_hid_emb = tf.zeros_like(hist_gru_emb[:,0,:])   # (32,14) 


In [20]:
hist_augru = AUGRU(item_dim+cate_dim)

In [21]:
for in_emb, in_att in zip(tf.transpose(hist_gru_emb, [1,0,2]),  # (20,32,14)
                                  tf.transpose(hist_attn, [2,0,1])):  # (20,32,1)
            hist_hid_emb = hist_augru(in_emb, hist_hid_emb, in_att)

In [22]:
hist_hid_emb

<tf.Tensor: id=2546, shape=(32, 14), dtype=float32, numpy=
array([[ 0.14648642,  0.13651685,  0.15864761, -0.17104676,  0.01430989,
         0.18607275, -0.01829286, -0.09990608,  0.005282  , -0.05237443,
         0.04945968,  0.11195529, -0.23012908,  0.09918956],
       [ 0.13782021,  0.14846215,  0.14881082, -0.1659787 ,  0.03232015,
         0.18659937,  0.00151965, -0.119367  ,  0.02819161, -0.04637884,
         0.03524139,  0.11855108, -0.24317914,  0.07702108],
       [ 0.00978375,  0.04397272,  0.06917825, -0.10100146,  0.0051847 ,
         0.08837738, -0.0102619 , -0.03727499, -0.02951335, -0.09204036,
         0.18127629,  0.12526385, -0.13742772,  0.1396219 ],
       [ 0.1591132 ,  0.06857026,  0.0869699 , -0.16727331,  0.01361358,
         0.16379169,  0.00281742, -0.08893263,  0.02659938,  0.00947718,
         0.07453416,  0.06415336, -0.22427154,  0.02158549],
       [ 0.14735484,  0.05605894,  0.12703943, -0.19016394, -0.06699548,
         0.16076122, -0.02679097, -0.130

In [23]:
 join_emb = tf.concat([user_emb, item_join_emb, hist_hid_emb], -1)

In [24]:
join_emb

<tf.Tensor: id=2549, shape=(32, 42), dtype=float32, numpy=
array([[ 0.85401475,  0.17966537,  0.6078861 , ...,  0.11195529,
        -0.23012908,  0.09918956],
       [ 0.5997213 ,  0.82931554,  0.59010124, ...,  0.11855108,
        -0.24317914,  0.07702108],
       [ 0.9350175 ,  0.7345248 ,  0.08661092, ...,  0.12526385,
        -0.13742772,  0.1396219 ],
       ...,
       [ 0.5221897 ,  0.48507   ,  0.51957184, ...,  0.09880868,
        -0.1713708 ,  0.08626675],
       [ 0.06606931,  0.81279397,  0.529611  , ...,  0.11239174,
        -0.23035268,  0.10076143],
       [ 0.03532085,  0.32427463,  0.46540797, ...,  0.08970471,
        -0.1697708 ,  0.04844797]], dtype=float32)>

In [25]:
output = tf.squeeze(fc(join_emb)) 

In [26]:
output[1]

<tf.Tensor: id=2685, shape=(2,), dtype=float32, numpy=array([ 1.2612679, -1.0014291], dtype=float32)>

In [27]:
logit = tf.keras.activations.sigmoid(output)


In [30]:
loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
    labels=[1.0,1.0,0.0], logits=[0.6,0.7,0.2], name=None
))

In [31]:
tf.keras.metrics.Sum(loss)

In [32]:
tf.Variable(tf.zeros([5]), dtype=tf.float32)

<tf.Variable 'Variable:0' shape=(5,) dtype=float32, numpy=array([0., 0., 0., 0., 0.], dtype=float32)>

DICE

In [35]:
axis=-1
epsilon=0.000000001
_x = np.random.rand(32,14)
_x = tf.constant(_x)

In [36]:
_x[:2]

<tf.Tensor: id=2737, shape=(2, 14), dtype=float64, numpy=
array([[0.18354011, 0.80482988, 0.72830186, 0.26298564, 0.69988818,
        0.49241716, 0.40259916, 0.3139576 , 0.77811343, 0.08418105,
        0.60443597, 0.21147113, 0.0755041 , 0.93938512],
       [0.86974583, 0.63982098, 0.00433048, 0.45329378, 0.76117299,
        0.94724667, 0.83008082, 0.65483281, 0.51133978, 0.32771291,
        0.1799161 , 0.05013963, 0.13186533, 0.95416694]])>

In [41]:
reduction_axes = list(range(len(_x.get_shape())))
reduction_axes

[0, 1]

In [42]:
del reduction_axes[axis]
reduction_axes

[0]

In [43]:
broadcast_shape = [1] * len(_x.get_shape())
broadcast_shape[axis] = 14

In [44]:
broadcast_shape

[1, 14]

In [45]:
mean = tf.reduce_mean(_x, axis=reduction_axes)
mean

<tf.Tensor: id=2739, shape=(14,), dtype=float64, numpy=
array([0.44129831, 0.48227416, 0.40288465, 0.53571309, 0.50323327,
       0.57323386, 0.48644189, 0.54528739, 0.4613733 , 0.44390142,
       0.54202612, 0.53564627, 0.46645622, 0.49391055])>

In [61]:
brodcast_mean = tf.reshape(mean, broadcast_shape)
std = tf.reduce_mean(tf.square(_x - brodcast_mean) + epsilon, axis=reduction_axes)
std = tf.sqrt(std)
brodcast_std = tf.reshape(std, broadcast_shape)

<tf.Tensor: id=2835, shape=(1, 14), dtype=float64, numpy=
array([[0.28816411, 0.29212979, 0.29721676, 0.26816005, 0.23223909,
        0.28133979, 0.26817694, 0.28613305, 0.28368718, 0.32460106,
        0.29742075, 0.27289657, 0.32352745, 0.30904081]])>

In [ ]:
x_normed = self.bn(_x)
x_p = tf.keras.activations.sigmoid(self.beta * x_normed)


alphas = tf.Variable(tf.zeros([14]), dtype=tf.float32)
alphas


alphas * (1.0 - x_p) * _x + x_p * _x